# Magic with OpenCV and xwidges running in Jupyter Notebook (xeus-cling kernel)

[Inspiration from the xwidgets binder](https://mybinder.org/v2/gh/jupyter-xeus/xwidgets/stable?filepath=notebooks/xwidgets.ipynb)

In [1]:
#include <jupyter/opencv>
#include <opencv2/core.hpp>
#include <opencv2/imgproc.hpp>

using namespace cv;

In [2]:
inline bool isPowerOf2(const size_t _Value) {
    return (_Value != 0 && (_Value & (_Value - 1)) == 0);
}


In [3]:
inline void shiftFromCornerToCenter(cv::Mat& src) {
    CV_Assert_2((src.cols & 1) == 0, (src.rows & 1) == 0);
    int halfWidth = src.cols / 2;
    int halfHeight = src.rows / 2;
    cv::Mat leftTop = src(cv::Rect(0, 0, halfWidth, halfHeight));
    cv::Mat rightTop = src(cv::Rect(halfWidth, 0, halfWidth, halfHeight));
    cv::Mat leftBottom = src(cv::Rect(0, halfHeight, halfWidth, halfHeight));
    cv::Mat rightBottom = src(cv::Rect(halfWidth, halfHeight, halfWidth, halfHeight));
    cv::Mat temp;
    // Swap leftTop with rightBottom
    rightBottom.copyTo(temp);
    leftTop.copyTo(rightBottom);
    temp.copyTo(leftTop);
    // Swap rightTop with leftBottom
    rightTop.copyTo(temp);
    leftBottom.copyTo(rightTop);
    temp.copyTo(leftBottom);
}


In [4]:
inline cv::Mat makeRGB(const cv::Mat& image) {
    if (image.type() == CV_8UC3) {
        return image;
    }

    cv::Mat rgbImage;
    if (image.type() == CV_8UC1) {
        cvtColor(image, rgbImage, cv::COLOR_GRAY2RGB);
    }
    else if (image.type() == CV_32FC1) {
        image.convertTo(rgbImage, CV_8U, 255.0);
        cvtColor(rgbImage, rgbImage, cv::COLOR_GRAY2RGB);
    }
    return rgbImage;
}


In [5]:
inline cv::Mat dftFilter(const cv::Mat& src, const cv::Mat& filter) {
    CV_Assert(src.size() == filter.size());
    CV_Assert_2(isPowerOf2(src.cols), isPowerOf2(filter.rows));
    CV_Assert_2(src.type() == CV_32F, filter.type() == CV_32F);

    cv::Mat matWithZeroes = cv::Mat::zeros(src.size(), CV_32F);
    cv::Mat complexInput1, complexOutput1;
    {
        cv::Mat planes[] = { src,  matWithZeroes };
        merge(planes, 2, complexInput1);
    }
    dft(complexInput1, complexOutput1, cv::DFT_SCALE | cv::DFT_COMPLEX_OUTPUT | cv::DFT_COMPLEX_INPUT);

    cv::Mat	complexFilter;
    {
        cv::Mat planes[] = { filter,  filter };
        merge(planes, 2, complexFilter);
    }

    shiftFromCornerToCenter(complexFilter);

    cv::Mat frequencyDomainResult(src.size(), CV_32FC2);
    mulSpectrums(complexOutput1, complexFilter, frequencyDomainResult, 0);

    cv::Mat finalImage;
    idft(frequencyDomainResult, finalImage, cv::DFT_COMPLEX_OUTPUT);

    // Calculate the magnitude
    {
        cv::Mat planes[] = { cv::Mat(finalImage.size(), CV_32F), cv::Mat(finalImage.size(), CV_32F) };
        split(finalImage, planes);                   // planes[0] = Re(DFT(I), planes[1] = Im(DFT(I))
        magnitude(planes[0], planes[1], finalImage); // finalImage = magnitude
    }
    
    return finalImage;
}


In [6]:
inline void displayImages(const Mat& img1, const Mat& img2) {
    const int border = 10;
    Size canvasSize(img1.cols + img2.cols + border, max(img1.rows, img2.rows));
    cv::Mat canvas(canvasSize, CV_8UC3);
    canvas.setTo(Scalar::all(255));

    cv::Mat rgbImage = makeRGB(img1);
    rgbImage.copyTo(canvas(Rect(0, 0, img1.cols, img1.rows)));
    rgbImage = makeRGB(img2);

    rgbImage.copyTo(canvas(Rect(img1.cols + border, 0, img2.cols, img2.rows)));
    im::mat_show(canvas);
}

In [7]:
#include <jupyter/opencv>
#include <opencv2/core.hpp>
#include <opencv2/imgproc.hpp>

const int imageSize = 1024;

Mat hummingbirdImage = imread("hb_1024.png", IMREAD_GRAYSCALE);
hummingbirdImage.convertTo(hummingbirdImage, CV_32FC1, 1.0 / 255.0);
resize(hummingbirdImage, hummingbirdImage, Size(imageSize, imageSize));

## $$ f_k = \sum_{n=0}^{N-1}{s_n\cdot e^{-ikn\frac{2\pi}{N}}} = \sum_{n=0}^{N-1}{s_n[cos(kn{\frac{2\pi}{N}}) -i\cdot sin(kn{\frac{2\pi}{N}})]}$$

# An interactive demonstration of image filtering with Discrete Fourier Transform
Select below the range of frequencies to be kept and click **Update**

In [8]:
#include "xwidgets/xoutput.hpp"

xw::output out;

In [9]:
#include "xwidgets/xslider.hpp"

xw::slider<double> slider_01;
slider_01.max = 100;
slider_01.value = 16;
slider_01.style().handle_color = "blue";
slider_01.orientation = "horizontal";
slider_01.description = "Radius";

In [10]:
#include "xwidgets/xbutton.hpp"

xw::button bt_01;
bt_01.description = "Update";

In [11]:
#include <xcpp/xdisplay.hpp>

{
    using namespace cv;
    
    auto func = [&](){
        // Using a scope guard to enable output capture
        auto g = out.guard();
        // Clear the previous output
        xcpp::clear_output();

        Mat filter = Mat::zeros(hummingbirdImage.size(), CV_32F);
        double radius = slider_01.value();
        
        circle(filter, Point(filter.cols/2, filter.rows/2), radius, Scalar::all(1.0), -1, 8, 0);
        Mat filteredImage = dftFilter(hummingbirdImage, filter);
        displayImages(filter, filteredImage);
        
        xcpp::display(slider_01);
        xcpp::display(bt_01);
    };
    
    bt_01.on_click(func);
    
    func();
}

xcpp::display(out);

A Jupyter widget with unique id: c92c798da63a4167a6f13aea6219b3b6